In [ ]:
from datetime import datetime
import os
import requests
import zipfile
from bs4 import BeautifulSoup
import re

session_cookie = 'ditbt0fhh52pl7tnwhzchsriz8ahzd27'
RFNSA_No = '3922002'

def download_file_from_google_colab(output_file_path):
    from google.colab import files
    files.download(output_file_path)
    print(f"File downloaded: {output_file_path}")

def start_Session():
    """
    Start the session reusing the underlying TCP connection for multiple requests as
    well as with gzip compression and randomly selected useragent 
    """
    session = requests.Session()
    session.headers.update({'Accept-Encoding': 'gzip'})
    return session

session = start_Session()
# Send a GET request to the URL
response = session.get('https://www.rfnsa.com.au/'+ RFNSA_No +'/documents', headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                        'Cookie': 'sessionid='+session_cookie})

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile('.*var rfnsaSid = "(.*)"')
    script = soup.find_all('script', string=pattern)
    sid = pattern.match(script[0].text.replace('\n','')).group(1)
    

    projects_docs = soup.findAll("tr", {'class':['test-project-doc']})
    site_docs = soup.findAll("tr", {'class':['test-site-doc']})
    
    for p in range(0, len(projects_docs)):
        project_date = projects_docs[p].find_all("td")[0].get_text(strip=True)
        carrier = projects_docs[p].find_all("td")[1].get_text(strip=True)
        project_name = projects_docs[p].find_all("td")[2].get_text(strip=True).replace(carrier,'').replace('Co.','').strip()
        System = projects_docs[p].find_all("td")[3].get_text(strip=True)
        id_documents = projects_docs[p].find_all("td")[4].find('a')['data-target'].replace('#','')
        #documents = soup.find("div", {"id": id_documents}).find_all("tr", class_="test-project-inner-doc clickable-row")
        documents = soup.find("div", {"id": id_documents}).find_all("tr", {'class':['test-project-inner-doc clickable-row']})

        for d in range(0,len(documents)):
            document_name = documents[d].find_all('td')[0].get_text(separator = '\n', strip = True).split('\n')[1]
            upload_date = datetime.strptime(documents[d].find_all('td')[2].text, '%d %b %Y %H:%M:%S').strftime('%Y-%m-%d')
            document_url = 'https://www.rfnsa.com.au'+ documents[d].find_all('a')[1]['data-url'].replace('embed','download').replace('-pdf','') + '?target=blank&sid=' + sid
            #print(document_url)
            file_name =  f"{RFNSA_No}_{carrier}_{project_name}_{upload_date.replace(' ', '_')}_{document_name}"

            document_response = session.get(document_url, headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                            'Cookie': session_cookie})  
            if document_response.status_code == 200:
                    # Save the document with the new name
                    with open(file_name, "wb") as file:
                        file.write(document_response.content)
                        if 'google.colab' in str(get_ipython()):
                            download_file_from_google_colab(file_name)
                    print(f"Downloaded: {file_name}")
            else:
                print(f"Failed to download: {document_url}")
                
    for p in range(0, len(site_docs)):
        project_date = site_docs[p].find_all("td")[0].get_text(strip=True)
        
        carrier = site_docs[p].find_all("td")[1].get_text(strip=True)
        if carrier:
            carrier = '_' + carrier
            
        project_name = site_docs[p].find_all("td")[2].get_text(strip=True).replace(carrier,'').replace('Co.','').strip()
        if project_name:
            project_name = '_' + project_name
            
        System = site_docs[p].find_all("td")[3].get_text(strip=True)
        id_documents = site_docs[p].find_all("td")[4].find('a')['data-target'].replace('#','')
        documents = soup.find("div", {"id": id_documents}).select('tr.test-site-inner-doc.clickable-row')

        for d in range(0,len(documents)):
            document_name_custom = documents[d].find_all('td')[0].get_text(separator = '\n', strip = True).split('\n')[0]
            document_name = documents[d].find_all('td')[0].get_text(separator = '\n', strip = True).split('\n')[1]
            
            for s in re.split('[_| .]+',document_name):
                document_name_custom2 = re.sub(s, '', document_name_custom, flags=re.I)
                document_name_custom = document_name_custom2.strip()
            document_name_custom = " ".join(document_name_custom2.split())
            
            upload_date = datetime.strptime(documents[d].find_all('td')[2].text, '%d %b %Y %H:%M:%S').strftime('%Y-%m-%d')
            document_url = 'https://www.rfnsa.com.au'+ documents[d].select('a.download-document')[0]['data-url']
            #print(document_url)
            file_name =  f"{RFNSA_No}{carrier}{project_name}_{upload_date.replace(' ', '_')}_{document_name_custom}_{document_name}"

            document_response = session.get(document_url, headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                            'Cookie': session_cookie})  
            if document_response.status_code == 200:
                    # Save the document with the new name
                    with open(file_name, "wb") as file:
                        file.write(document_response.content)
                        if 'google.colab' in str(get_ipython()):
                            download_file_from_google_colab(file_name)
                    print(f"Downloaded: {file_name}")
            else:
                print(f"Failed to download: {document_url}")


In [ ]:
# Check if the request was successful
if response.status_code == 200:    

    projects_img = soup.findAll("tr", {'class':['test-project-img', 'test-site-img']})
    
    for p in range(0, len(projects_img)):
        img_project_date = datetime.strptime(projects_img[p].find_all("td")[0].get_text(strip=True), '%d %b %Y').strftime('%Y-%m-%d')
        img_carrier = projects_img[p].find_all("td")[1].get_text(strip=True)
        if img_carrier:
            img_carrier = '_' + img_carrier
        
        img_project_name = projects_img[p].find_all("td")[2].get_text(strip=True).replace(img_carrier,'')
        if img_project_name:
            img_project_name = '_' + img_project_name
            
        img_System = projects_img[p].find_all("td")[3].get_text(strip=True)
        
        id_img = projects_img[p].find_all("td")[4].find('a')['data-target'].replace('#','')
        images = soup.find("div", {"id": id_img}).select('tr.test-site-inner-doc.clickable-row')
        images_dates_time_list = []
        images = soup.find("div", {"id": id_img}).select('table')[0]
        for j in images.find_all('tr')[1:]:
                    images_dates_time_list.extend([tv.text.strip() for tv in j.select('td:nth-of-type(4)')] )
        images_dates_list = []
        for x in list(dict.fromkeys(images_dates_time_list)):
            images_dates_list.append(datetime.strptime(x, '%d %b %Y %H:%M:%S').strftime('%Y-%m-%d'))
        latest_image_date = max(images_dates_list)
        
        img_url = 'https://www.rfnsa.com.au'+ soup.find("div", {"id": id_img}).find_all("a", class_="download-document")[0]['data-url']+'&sid=' + sid

        img_file_name =  f"{RFNSA_No}_{latest_image_date}{img_carrier}{img_project_name}.zip"
        img_response = session.get(img_url, headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                            'Cookie': session_cookie})  
        if img_response.status_code == 200:
                    # Save the document with the new name
                with open(img_file_name, "wb") as file:
                    file.write(img_response.content)
                    if 'google.colab' in str(get_ipython()):
                        download_file_from_google_colab(img_file_name)
                print(f"Downloaded: {img_file_name}")
        else:
            print(f"Failed to download: {img_url}")


In [ ]:
 # Check if the document is a zip file
                if document_response.headers.get("Content-Type") == "application/zip":
                    zip_file_path = "temp.zip"
                    with open(zip_file_path, "wb") as zip_file:
                        zip_file.write(document_response.content)

                    # Unzip the file
                    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                        # Extract and save the files with the new names
                        for zip_info in zip_ref.infolist():
                            extracted_file_name = f"{carrier}_{project_name}_{upload_date.replace(' ', '_')}_{zip_info.filename}"
                            zip_ref.extract(zip_info, path=extracted_file_name)
                            if 'google.colab' in str(get_ipython()):
                                download_file_from_google_colab(extracted_file_name)

                    # Remove the original zip file
                    os.remove(zip_file_path)

                    print(f"Unzipped and copied to roof folder: {file_name}")
                else: